***The resume data is from: https://www.kaggle.com/datasets/snehaanbhawal/resume-dataset. I only retrieve 25 records from it. Labels can be seen in the code below--Skill, University, Degree, Job Description, Duration, Position, Experience.***

In [1]:
import spacy
from spacy import displacy
import random
import json
import logging

### 1. Load Annotated resume document by doccano

In [38]:
filename = 'resume.jsonl'

In [39]:
with open(filename,'r') as json_file:
    data=json.load(json_file)

In [6]:
data

{'id': 7,
 'text': '11847784\t"         HR SPECIALIST       Summary    Possess 15+ years of experience as an HR Classification/Compensation Lead and as an HR Specialist, Recruitment. Managed recruitment efforts that included qualifying candidates based on documented knowledge, skills, and abilities to match organizational resource needs. Ensured candidate fulfillment of the preemployment process; i.e., background checks, drug screens, and reference checks. Developed and implemented competency and skill-based pay programs for a variety of occupations including medical (nurses), mechanical (machine operators), and civil engineers. Led a statewide review of jobs (18,000 employees) toward determining appropriate FLSA designations (e.g., executive, administrative, computer-related, creative). Provided executive and middle management consultation toward acquiring class/levels and roles necessary to accomplish departmental goals and objectives that resulted in mission successes. Training and 

### 2. Convert annotated json file to spacy required format

In [8]:
del data['id']
data

{'text': '11847784\t"         HR SPECIALIST       Summary    Possess 15+ years of experience as an HR Classification/Compensation Lead and as an HR Specialist, Recruitment. Managed recruitment efforts that included qualifying candidates based on documented knowledge, skills, and abilities to match organizational resource needs. Ensured candidate fulfillment of the preemployment process; i.e., background checks, drug screens, and reference checks. Developed and implemented competency and skill-based pay programs for a variety of occupations including medical (nurses), mechanical (machine operators), and civil engineers. Led a statewide review of jobs (18,000 employees) toward determining appropriate FLSA designations (e.g., executive, administrative, computer-related, creative). Provided executive and middle management consultation toward acquiring class/levels and roles necessary to accomplish departmental goals and objectives that resulted in mission successes. Training and Developmen

In [70]:
def convert_doccano_to_spacy(data):
    try:
        training_data = []
        lines=[]
        text = data['text']
        entities = []
        for each in data['label']:
            points = each[:2]
            label = each[2:][0]
            if not isinstance(label, list):
                labels = [label]
            for label in labels:
                entities.append((points[0], points[1]+1 ,label))


        training_data.append((text, {"entities" : entities}))

        return training_data
    except Exception as e:
        logging.exception("Unable to process " + "\n" + "error = " + str(e))
        return None

In [71]:
TRAIN_DATA = convert_doccano_to_spacy(data)
TRAIN_DATA

[('11847784\t"         HR SPECIALIST       Summary    Possess 15+ years of experience as an HR Classification/Compensation Lead and as an HR Specialist, Recruitment. Managed recruitment efforts that included qualifying candidates based on documented knowledge, skills, and abilities to match organizational resource needs. Ensured candidate fulfillment of the preemployment process; i.e., background checks, drug screens, and reference checks. Developed and implemented competency and skill-based pay programs for a variety of occupations including medical (nurses), mechanical (machine operators), and civil engineers. Led a statewide review of jobs (18,000 employees) toward determining appropriate FLSA designations (e.g., executive, administrative, computer-related, creative). Provided executive and middle management consultation toward acquiring class/levels and roles necessary to accomplish departmental goals and objectives that resulted in mission successes. Training and Development: Eigh

### 3. NER model

##### *blank Language model*

In [65]:
nlp=spacy.blank('en')

##### *NER pipeline*

In [66]:
if 'ner' not in nlp.pipe_names:
    ner=nlp.create_pipe('ner')
    nlp.add_pipe(ner,last=True)

##### *Adding labels*

In [72]:
for _,annotations in TRAIN_DATA:
    for ent in annotations.get('entities'):
        ner.add_label(ent[2])

#####  *Get names of other pipes to disable them during training*

In [73]:
other_pipes=[pipe for pipe in nlp.pipe_names if pipe !='ner']

##### *Train only NER*

In [74]:
with nlp.disable_pipes(*other_pipes):
    optimizer = nlp.begin_training()
    for itn in range(100):
        print("Statring iteration " + str(itn))
        random.shuffle(TRAIN_DATA)
        losses = {}
        for text, annotations in TRAIN_DATA:
            nlp.update(
                [text],  # batch of texts
                [annotations],  # batch of annotations
                drop=0.1,  # dropout - make it harder to memorise data
                sgd=optimizer,  # callable to update weights
                losses=losses)
        print(losses)

        doc = nlp("02/2015   to   Current     HR Employee Relations Specialist.")
        print ("Entities= " + str(["" + str(ent.text) + "_" + str(ent.label_) for ent in doc.ents]))
        doc = nlp("Dedicated and self-motivated professional with experience in providing outstanding support to business partners. Skillful in tracking details, office management, and following-up with internal and external partners to ensure ontime completion of projects.")
        print ("Entities= " + str(["" + str(ent.text) + "_" + str(ent.label_) for ent in doc.ents]))

Statring iteration 0
{'ner': 16648.330108642578}
Entities= []
Entities= []
Statring iteration 1
{'ner': 16464.135345458984}
Entities= []
Entities= []
Statring iteration 2
{'ner': 16316.135833740234}
Entities= []
Entities= []
Statring iteration 3
{'ner': 15739.951950073242}
Entities= []
Entities= []
Statring iteration 4
{'ner': 14372.240753173828}
Entities= []
Entities= []
Statring iteration 5
{'ner': 11048.793273925781}
Entities= []
Entities= []
Statring iteration 6
{'ner': 7682.323257446289}
Entities= []
Entities= []
Statring iteration 7
{'ner': 4871.496940612793}
Entities= []
Entities= []
Statring iteration 8
{'ner': 4606.156135559082}
Entities= []
Entities= []
Statring iteration 9
{'ner': 4988.917922973633}
Entities= []
Entities= []
Statring iteration 10
{'ner': 5246.429626464844}
Entities= []
Entities= []
Statring iteration 11
{'ner': 5113.501792907715}
Entities= []
Entities= []
Statring iteration 12
{'ner': 4952.172225952148}
Entities= []
Entities= []
Statring iteration 13
{'ner':

In [82]:
#do prediction
doc = nlp("02/2015   to   Current     HR Employee Relations Specialist.")
print ("Entities= " + str(["" + str(ent.text) + "_" + str(ent.label_) for ent in doc.ents]))

doc = nlp("Dedicated and self-motivated professional with experience in providing outstanding support to business partners. Skillful in tracking details, office management, and following-up with internal and external partners to ensure ontime completion of projects.")
print ("Entities= " + str(["" + str(ent.text) + "_" + str(ent.label_) for ent in doc.ents]))

Entities= ['HR Employee Relations Specialist_Job Description']
Entities= ['experience in providing outstanding support to business partners. Skillful in tracking details, office management, and following-up with internal and external partners to ensure ontime completion of projects._Experience']


#### 4. Test

In [101]:
doc = nlp("Versatile  media professional with background in Communications, Marketing, Human Resources and Technology")
print ("Entities= " + str(["" + str(ent.text) + "_" + str(ent.label_) for ent in doc.ents]))

Entities= ['background in Communications, Marketing, Human Resources and Technology_Experience']


In [102]:
displacy.render(doc, jupyter=True, style='ent')